In [15]:
import pandas as pd


# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import dataproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# a. load PL data
pl = pd.read_csv('pl_all_seasons.csv')

# b. print PL data

print(pl)

print(pl.loc[:, 'FTR'])

# c. choose relevant data in df

df = pd.DataFrame(pl,  columns=['FTR', 'B365H', 'B365D', 'B365A'])

# d. add boolean of result Home, Draw, Away and then print df

df['H_true'] = df['FTR'].str.count("H")
df['D_true'] = df['FTR'].str.count("D")
df['A_true'] = df['FTR'].str.count("A")

print(df)

# e. multiply bolean of Draw (D_true) with B365 odds of draw to get each games return for you

return_D = ( df.loc[:, 'D_true'] * df.loc[:, 'B365D'] )

print(return_D)

# f. sum these individual returns to get you full season return

sum_return_D = return_D.sum()

print(sum_return_D)

#g. find the net return by subtracting the DKK 1 * 380 games = DKK 3800 you have betted during the season to find your gain/loss

net_return_D = sum_return_D - 3800

print(net_return_D)

#g. find the return in percentage by dividing you gain with the DKK 1 * 380 games = DKK 3800 you have betted during the season and subtract 1.

net_pct_returm_D = sum_return_D / 3800 - 1

print(net_pct_returm_D)


Div      Date     HomeTeam      AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0     E0  15/08/09  Aston Villa         Wigan     0     2   A     0     1   A   
1     E0  15/08/09    Blackburn      Man City     0     2   A     0     1   A   
2     E0  15/08/09       Bolton    Sunderland     0     1   A     0     1   A   
3     E0  15/08/09      Chelsea          Hull     2     1   H     1     1   D   
4     E0  15/08/09      Everton       Arsenal     1     6   A     0     3   A   
...   ..       ...          ...           ...   ...   ...  ..   ...   ...  ..   
3795  E0  12/05/19    Liverpool        Wolves     2     0   H     1     0   H   
3796  E0  12/05/19   Man United       Cardiff     0     2   A     0     1   A   
3797  E0  12/05/19  Southampton  Huddersfield     1     1   D     1     0   H   
3798  E0  12/05/19    Tottenham       Everton     2     2   D     1     0   H   
3799  E0  12/05/19      Watford      West Ham     1     4   A     0     2   A   

      ... BbMx>2.5  BbAv>2.5  Bb

In [17]:
# a. load PL data
pl = pd.read_csv('pl_all_seasons.csv')

# b. print PL data

print(pl)

print(pl.loc[:, 'FTR'])

# c. choose relevant data in df

df = pd.DataFrame(pl,  columns=['FTR', 'B365H', 'B365D', 'B365A'])

# d. add boolean of result Home, Draw, Away and then print df

df['H_true'] = df['FTR'].str.count("H")
df['D_true'] = df['FTR'].str.count("D")
df['A_true'] = df['FTR'].str.count("A")

print(df)

# e. multiply bolean of Draw (A_true) with B365 odds of draw to get each games return for you

return_A = ( df.loc[:, 'A_true'] * df.loc[:, 'B365A'] )

print(return_A)

# f. sum these individual returns to get you full season return

sum_return_A = return_A.sum()

print(sum_return_A)

#g. find the net return by subtracting the DKK 1 * 3800 games = DKK 3800 you have betted during the season to find your gain/loss

net_return_A = sum_return_A - 3800

print(net_return_A)

#g. find the return in percentage by dividing you gain with the DKK 1 * 3800 games = DKK 3800 you have betted during the season and subtract 1.

net_pct_returm_A = sum_return_A / 3800 - 1

print(net_pct_returm_A)


Div      Date     HomeTeam      AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0     E0  15/08/09  Aston Villa         Wigan     0     2   A     0     1   A   
1     E0  15/08/09    Blackburn      Man City     0     2   A     0     1   A   
2     E0  15/08/09       Bolton    Sunderland     0     1   A     0     1   A   
3     E0  15/08/09      Chelsea          Hull     2     1   H     1     1   D   
4     E0  15/08/09      Everton       Arsenal     1     6   A     0     3   A   
...   ..       ...          ...           ...   ...   ...  ..   ...   ...  ..   
3795  E0  12/05/19    Liverpool        Wolves     2     0   H     1     0   H   
3796  E0  12/05/19   Man United       Cardiff     0     2   A     0     1   A   
3797  E0  12/05/19  Southampton  Huddersfield     1     1   D     1     0   H   
3798  E0  12/05/19    Tottenham       Everton     2     2   D     1     0   H   
3799  E0  12/05/19      Watford      West Ham     1     4   A     0     2   A   

      ... BbMx>2.5  BbAv>2.5  Bb

In [18]:
# a. load PL data
pl = pd.read_csv('pl_all_seasons.csv')

# b. print PL data

print(pl)

print(pl.loc[:, 'FTR'])

# c. choose relevant data in df

df = pd.DataFrame(pl,  columns=['FTR', 'B365H', 'B365D', 'B365A'])

# d. add boolean of result Home, Draw, Away and then print df

df['H_true'] = df['FTR'].str.count("H")
df['D_true'] = df['FTR'].str.count("D")
df['A_true'] = df['FTR'].str.count("A")

print(df)

# e. multiply bolean of Draw (H_true) with B365 odds of draw to get each games return for you

return_H = ( df.loc[:, 'H_true'] * df.loc[:, 'B365H'] )

print(return_H)

# f. sum these individual returns to get you full season return

sum_return_H = return_H.sum()

print(sum_return_H)

#g. find the net return by subtracting the DKK 1 * 3800 games = DKK 3800 you have betted during the season to find your gain/loss

net_return_H = sum_return_H - 3800

print(net_return_H)

#g. find the return in percentage by dividing you gain with the DKK 1 * 3800 games = DKK 3800 you have betted during the season and subtract 1.

net_pct_returm_H = sum_return_H / 3800 - 1

print(net_pct_returm_H)


Div      Date     HomeTeam      AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0     E0  15/08/09  Aston Villa         Wigan     0     2   A     0     1   A   
1     E0  15/08/09    Blackburn      Man City     0     2   A     0     1   A   
2     E0  15/08/09       Bolton    Sunderland     0     1   A     0     1   A   
3     E0  15/08/09      Chelsea          Hull     2     1   H     1     1   D   
4     E0  15/08/09      Everton       Arsenal     1     6   A     0     3   A   
...   ..       ...          ...           ...   ...   ...  ..   ...   ...  ..   
3795  E0  12/05/19    Liverpool        Wolves     2     0   H     1     0   H   
3796  E0  12/05/19   Man United       Cardiff     0     2   A     0     1   A   
3797  E0  12/05/19  Southampton  Huddersfield     1     1   D     1     0   H   
3798  E0  12/05/19    Tottenham       Everton     2     2   D     1     0   H   
3799  E0  12/05/19      Watford      West Ham     1     4   A     0     2   A   

      ... BbMx>2.5  BbAv>2.5  Bb